In [ ]:
!unzip data.zip

In [2]:
import torch
import torchvision
import pandas as pd

from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [3]:
batch_size = 32
lerning_rate = 0.001
epoch = 5

In [ ]:
sanders_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = ImageFolder("/content/data/train/images", sanders_transform)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4)

table = pd.read_csv("data/train/images.csv")

In [7]:
labels = [torch.Tensor([table['count'][j] for j in range(batch_size if table['file_name'].size % batch_size * i == 0 else batch_size * i - table['file_name'].size)]) for i in range(table['file_name'].size // batch_size)]

In [8]:
labels

# нужно сделать срез numpy.array_split

[tensor([6., 5., 2., 4., 3., 7., 4., 7., 7., 1., 4., 5., 3., 3., 3., 4., 1., 6.,
         4., 6., 2., 4., 3., 1., 7., 6., 6., 1., 2., 3., 3., 4.]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor([]),
 tensor

In [26]:
class BurgerNet(nn.Module):
    def __init__(self):
         super(BurgerNet, self).__init__()
         self.layer1 = nn.Sequential( nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
         self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
         self.drop_out = nn.Dropout()
         self.fc1 = nn.Linear(7 * 7 * 64, 1000)
         self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.view(out.size(0), -1)
      out = self.drop_out(out)
      out = self.fc1(out)
      out = self.fc2(out)

      return out

model = BurgerNet()
criterion = nn.CrossEntropyLoss()
mac_optimizer = torch.optim.Adam(model.parameters(), lr=lerning_rate)

In [ ]:
loss_list = []
acc_list = []

for epoch_num in range(epoch):
  for i, (images, labels) in enumerate(data_loader):
    print(labels.size())
    outputs = model(images)

